In [2]:
import numpy as np

%cd -q ..

import utils.constants as const
from ../../atoms.atomic_species import *

import matplotlib.pyplot as plt
%matplotlib inline

SyntaxError: invalid syntax (<ipython-input-2-af08c4fee3cc>, line 6)

In [ ]:
_Zelem = {'H':1, 'He':2, 'Li':3, 'Be':4, 'B':5, 'C':6, 'N':7,
              'O':8, 'F':9,'Ne':10, 'Na':11, 'Mg':12, 'Al':13,
              'Si':14, 'S':16, 'Ar':18, 'Ca':20, 'Fe':26}

class element:
    def __init__(self, name, ppm=None, E_th=None):
        self.name = name
        self.ppm = ppm
        self.E_th = E_th
        self.Z = _Zelem[name]
        self.absorbers = [None,]*(self.Z)
        for i in range(self.Z):
            self.absorbers[i] = atomic_species(self.name+' '
                                               +arabic_to_roman(i+1))
        if E_th is not None:
            for absorber in self.absorbers:
                if absorber.verner_data['E_th'] < E_th:
                    self.absorbers.remove(absorber)
                    
                    
    def get_max_sigma(self, hnu):
        self.max_sigma = np.zeros(len(hnu))
        for absorber in self.absorbers:
            self.max_sigma = np.maximum.reduce(
                [self.max_sigma, absorber.cross_section(hnu, valid_range=False)]
            )

In [ ]:
# solar elements above 100 ppm
H_ppm = 909964
He_ppm = 88714
O_ppm = 477
C_ppm = 326
N_ppm = 102
Ne_ppm = 100

hydrogen = element('H', ppm=H_ppm)
E_th = hydrogen.absorbers[0].verner_data['E_th']
helium = element('He', ppm=He_ppm, E_th=E_th)
oxygen = element('O', ppm=O_ppm, E_th=E_th)
carbon = element('C', ppm=C_ppm, E_th=E_th)
nitrogen = element('N', ppm=N_ppm, E_th=E_th)
neon = element('Ne', ppm=Ne_ppm, E_th=E_th)

solar_elements = [hydrogen, helium, oxygen, carbon, nitrogen, neon]

In [ ]:
hnu = np.linspace(E_th, 1e3, 1024)
wl = const.hc/(const.eV*hnu)
for elm in solar_elements:
    elm.get_max_sigma(hnu)

In [ ]:
fig, ax = plt.subplots()

norm = hydrogen.ppm*hydrogen.max_sigma
# Ntot = 1e6/(norm)
# axB = ax.twinx()
# axB.plot(hnu, Ntot, '--')
# axB.set_yscale('log')

for elm in solar_elements:
    l = elm.ppm*elm.max_sigma/norm
    l[l==0] = np.nan
    ax.plot(wl*1e7, l, label=elm.name)

ax.legend()
# ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylabel(r'$X_E \sigma_{\nu,E}^{max}/X_H \sigma_{\nu,H}$')
ax.set_xlabel('Wavelength (nm)')
ax.set_xlim([10, 90])
fig.tight_layout()